<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/geehydro/tree/master/examples/naip.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/geehydro/blob/master/examples/naip.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://mybinder.org/v2/gh/giswqs/geehydro/master?filepath=examples/naip.ipynb"><img width=58px src="https://mybinder.org/static/images/logo_social.png" />Run in binder</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/geehydro/blob/master/examples/naip.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

## Install Earth Engine API
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geehydro](https://github.com/giswqs/geehydro). The **geehydro** Python package builds on the [folium](https://github.com/python-visualization/folium) package and implements several methods for displaying Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, `Map.centerObject()`, and `Map.setOptions()`.
The magic command `%%capture` can be used to hide output from a specific cell.

In [1]:
# %%capture
# !pip install earthengine-api
# !pip install geehydro

Import libraries

In [2]:
import ee
import folium
import geehydro
from geehydro.utilities import *

Authenticate and initialize Earth Engine API. You only need to authenticate the Earth Engine API once. Uncomment the line `ee.Authenticate()` 
if you are running this notebook for this first time or if you are getting an authentication error.  

In [3]:
# ee.Authenticate()
ee.Initialize()

## Create an interactive map 
This step creates an interactive map using [folium](https://github.com/python-visualization/folium). The default basemap is the OpenStreetMap. Additional basemaps can be added using the `Map.setOptions()` function. 
The optional basemaps can be `ROADMAP`, `SATELLITE`, `HYBRID`, `TERRAIN`, or `ESRI`.

In [4]:
Map = folium.Map(location=[40, -100], zoom_start=4)
Map.setOptions('HYBRID')

## Add Earth Engine Python script 

In [5]:
#Step 1: Get county boundary
counties = ee.FeatureCollection('TIGER/2018/Counties')
# Map.addLayer(counties, {}, 'US Counties', False)

knox = counties.filter(ee.Filter.And(ee.Filter.eq('NAME', 'Knox'), ee.Filter.eq('STATEFP', '47')))
Map.centerObject(knox, 10)
Map.addLayer(knox, {}, 'Knox County')

naip_images = findNAIP(knox)
naip_count = naip_images.size().getInfo()
print('Available NAIP images: {}'.format(naip_count))

Available NAIP images: 4


In [6]:
dates = ee.List(naip_images.aggregate_array('system:time_start')).map(lambda x: ee.Date(x).format('YYYY-MM-dd'))
dates_list = dates.getInfo()
print(dates_list)

ndvi_vis = {
    'min': 0,
    'max': 1,
    'palette': [
      'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
      '74A901', '66A000', '529400', '3E8601', '207401', '056201',
      '004C00', '023B01', '012E01', '011D01', '011301']    
}

for i in range(0, naip_count):
    naip_image = ee.Image(naip_images.toList(naip_count).get(i))
    ndvi_image = naip_image.select('ndvi')
    Map.addLayer(naip_image, {'bands': ['N', 'R', 'G']}, 'NAIP ' + dates_list[i])
    Map.addLayer(ndvi_image, ndvi_vis, 'NDVI ' + dates_list[i])

['2010-04-11', '2012-05-28', '2014-04-27', '2016-04-18']


## Display Earth Engine data layers 

In [7]:
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map